# Analyze residential property assessment appeals for 2022S and 2023

In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [2]:
db = create_engine('postgresql://jon@localhost:5432/propertydb')

### Appeals for 2015-21

In [3]:
# https://data.wprdc.org/datastore/dump/67b18589-7bf5-4ca5-8767-3bceb318522c?q=&sort=_id+asc&fields=PARCEL+ID,TAX+YEAR,CLASS,CLASS+GROUP,TAX_STATUS,MUNI_CODE,MUNI_NAME,SCHOOL_CODE,SCHOOL_DISTRICT,COMPLAINANT&filters={"TAX+YEAR":+["2015",+"2016",+"2017",+"2018",+"2019",+"2020",+"2021"]}&format=csv
wprdcappeals = pd.read_csv('input/appeals-2015-21.csv')

In [4]:
wprdcappeals = wprdcappeals[(wprdcappeals.CLASS == 'RESIDENTIAL') & (wprdcappeals.TAX_STATUS == 'TAXABLE')]
wprdcappeals = wprdcappeals.rename(columns={'TAX YEAR': 'appealperiod', 'COMPLAINANT': 'filer'})

In [5]:
wprdcappeals = pd.pivot_table(wprdcappeals, index='appealperiod', columns='filer', values='PARCEL ID', aggfunc='count')
wprdcappeals = wprdcappeals.fillna(0)

In [6]:
wprdcappeals['MUNICIPALITY'] = wprdcappeals['Muni/School'] + wprdcappeals['Municipality'] + wprdcappeals['O/M/S'] + wprdcappeals['Owner/Muni']
wprdcappeals['PROPERTY OWNER'] = wprdcappeals['O/M/S'] + wprdcappeals['Owner'] + wprdcappeals['Owner/Muni'] + wprdcappeals['Owner/School']
wprdcappeals['SCHOOL DISTRICT'] = wprdcappeals['Muni/School'] + wprdcappeals['O/M/S'] + wprdcappeals['Owner/School'] + wprdcappeals['School District']

wprdcappeals = wprdcappeals[[
    'MUNICIPALITY',
    'PROPERTY OWNER',
    'SCHOOL DISTRICT'
]]

In [7]:
wprdcappeals

filer,MUNICIPALITY,PROPERTY OWNER,SCHOOL DISTRICT
appealperiod,,,
2015,1342.0,3015.0,2606.0
2016,48.0,2816.0,3506.0
2017,18.0,2420.0,3724.0
2018,6.0,2054.0,3758.0
2019,8.0,1868.0,5253.0
2020,133.0,1523.0,5703.0
2021,243.0,1219.0,8075.0


### Appeals for 2022R, 2022S and 2023

In [8]:
dbappeals = pd.read_sql_query("""SELECT appeals.*
FROM appeals
JOIN assessments
ON appeals.parcelid = assessments.parcelid
WHERE appeals.type IN ('R', 'R-RESIDENTIAL')
AND appeals.appealperiod IN ('2022R', '2022S', '2023')
AND assessments.taxcode = 'T';""", db)

In [9]:
dbappeals = pd.pivot_table(dbappeals, index='appealperiod', columns='filer', values='parcelid', aggfunc='count')

### Merge all appeals together, save to CSV

In [10]:
allappeals = pd.concat([wprdcappeals, dbappeals])

In [11]:
allappeals

filer,MUNICIPALITY,PROPERTY OWNER,SCHOOL DISTRICT
appealperiod,,,
2015,1342.0,3015.0,2606.0
2016,48.0,2816.0,3506.0
2017,18.0,2420.0,3724.0
2018,6.0,2054.0,3758.0
2019,8.0,1868.0,5253.0
2020,133.0,1523.0,5703.0
2021,243.0,1219.0,8075.0
2022R,216.0,882.0,10545.0
2022S,1.0,4478.0,1312.0


In [12]:
allappeals.to_csv('output/residential-filers.csv')